In [2]:
import pandas as pd
from datetime import datetime, timedelta
pd.options.mode.copy_on_write = True 

In [3]:
#Lit le fichier des gares et filtrent sur les gares qui accueillent des voyageurs
liste_city = pd.read_csv("./data_cleaned/stop_location_infos.csv", sep=',')

In [4]:
#Lit le fichier de stops
liste_stops_ter = pd.read_csv("./export-ter-gtfs-last/stops.csv", sep=",")
liste_stops_ter = liste_stops_ter[[str.strip(x[:18]) == "StopPoint:OCETrain" for x in liste_stops_ter["stop_id"]]]
liste_stops_ter = pd.merge(liste_stops_ter, liste_city, how="left", on="stop_id")


In [5]:
# Lit le fichier des temps d'arrêts des trains
liste_stop_times_ter = pd.read_csv("./export-ter-gtfs-last/stop_times.csv", sep=",")

liste_stop_times_ter = liste_stop_times_ter[[str.strip(str(x)[:18]) == "StopPoint:OCETrain" for x in liste_stop_times_ter["stop_id"]]]
liste_stop_times_ter = pd.merge(liste_stop_times_ter, liste_stops_ter[["stop_id", "city"]], how="inner", on="stop_id")


In [6]:
liste_stop_times_ter.columns

Index(['trip_id', 'arrival_time', 'departure_time', 'stop_id', 'stop_sequence',
       'stop_headsign', 'pickup_type', 'drop_off_type', 'shape_dist_traveled',
       'city'],
      dtype='object')

In [7]:
liste_trips_ter = pd.read_csv("./export-ter-gtfs-last/trips.csv", sep=",")
liste_calendar_ter = pd.read_csv("./export-ter-gtfs-last/calendar_dates.csv", sep=",")
print(len(liste_trips_ter))

liste_trips_ter = pd.merge(liste_trips_ter, liste_calendar_ter[["service_id", "date"]], how="inner", on="service_id")

print(len(liste_trips_ter))

liste_stop_times_ter = pd.merge(liste_stop_times_ter, liste_trips_ter[["date", "trip_id"]], how="inner", on="trip_id")

print(len(liste_stop_times_ter))

52753
819405
5955580


In [9]:
liste_stop_times_ter["date"] = [datetime(int(str(row.date)[:4]), int(str(row.date)[4:6]),  int(str(row.date)[-2:]), 0 if int(str(row.arrival_time).split(":")[0]) >= 24 else int(str(row.arrival_time).split(":")[0]), int(str(row.arrival_time).split(":")[1])) for row in liste_stop_times_ter.itertuples()]

In [10]:
liste_stop_times_ter.date.head(5)

0   2024-11-18 05:00:00
1   2024-11-18 05:08:00
2   2024-11-18 05:30:00
3   2024-11-18 05:40:00
4   2024-11-18 05:49:00
Name: date, dtype: datetime64[ns]

In [49]:
def check_destination(destination, layer):
    return len(layer.loc[layer["city"] == destination]) > 0

def path_back_propagation(layers, destination):
    paths = []
    city_to_fetch = destination
    for layer in reversed(layers):
        data = layer.loc[layer["city"] == city_to_fetch]
        if len(data) == 0:
            return "error"
        data = data.iloc[0]
        paths.append(data['current_path'])
        if data["previous_path"] == "":
            return paths[::-1]
        city_to_fetch = data["previous_path"].split(';')[0]

def init_row_list():
    return {
        'trip_id': [],
        'arrival_time': [],
        'departure_time': [],
        'stop_sequence': [],
        'stop_id': [],
        'stop_headsign': [],
        'pickup_type': [],
        'drop_off_type': [],
        'shape_dist_traveled': [],
        'city': [],
        'date': [],
        'previous_path': [],
    }

def get_format_data_to_list(row_list, row):
    row_list["trip_id"].append(row.trip_id)
    row_list["arrival_time"].append(row.arrival_time)
    row_list["departure_time"].append(row.departure_time)
    row_list["stop_id"].append(row.stop_id)
    row_list["stop_sequence"].append(row.stop_sequence)
    row_list["stop_headsign"].append(row.stop_headsign)
    row_list["pickup_type"].append(row.pickup_type)
    row_list["drop_off_type"].append(row.drop_off_type)
    row_list["shape_dist_traveled"].append(row.shape_dist_traveled)
    row_list["city"].append(row.city)
    row_list["date"].append(row.date)
    row_list["previous_path"].append(row.previous_path)
    return row_list
    

def filter_direction_trip(data, namesList, start_date=False):
    find_name = False
    trip_id = ""
    end_date = ""
    default_value = False
    row_list = init_row_list()

    if start_date != False:
        end_date = start_date + timedelta(days=1)
        default_value = True

    data = data.sort_values(by=["trip_id", "stop_sequence"])

    for row in data.itertuples():
        if not default_value:
            date_str = row.previous_path.split(';')[-1]
            date = datetime.strptime(date_str, "%d/%m/%Y, %H:%M:%S")
            start_date = date + timedelta(minutes=5)
            end_date = start_date + timedelta(days=1)

        if row.trip_id != trip_id:
            trip_id = row.trip_id
            find_name = False

        if row.city in namesList :#and (row.date >= start_date and row.date <= end_date):
            find_name = True

        if find_name:
            row_list = get_format_data_to_list(row_list, row)
 
    return pd.DataFrame(row_list)

def shortest_path(depart, destination, start_date):
    layers = []
    not_founded = 0 # vérifie qu'on a pas trouvé un chemin vers l'arrivée

    start_date = datetime.strptime(start_date, "%Y-%m-%d %H:%M:%S")
    end_date = start_date + timedelta(days=1)
    date_mask = (liste_stop_times_ter["date"] > start_date) & (liste_stop_times_ter["date"] < end_date)

    trips_id = set(liste_stop_times_ter.loc[(liste_stop_times_ter["city"] == depart) & date_mask]["trip_id"])
    layer = liste_stop_times_ter.loc[(liste_stop_times_ter["trip_id"].isin(trips_id)) & date_mask]
    
    layer["previous_path"] = ""
    layer = filter_direction_trip(layer, [depart], start_date)  
    layer = layer.sort_values(by="date").drop_duplicates(subset=["stop_id"])

    
    layer["current_path"] = layer["trip_id"] + ";" + layer["city"] + ";" + layer["date"].dt.strftime("%d/%m/%Y, %H:%M:%S")

    layers.append(layer)

    while not_founded < 100:
        if check_destination(destination, layers[-1]):
            print(path_back_propagation(layers, destination))
            return "destination trouver"
        
        filtered_data = liste_stop_times_ter.loc[(liste_stop_times_ter["city"].isin(layers[-1]["city"].unique())) & date_mask]

        paths = {}
        for row in filtered_data.itertuples():
            paths[row.trip_id] = str(row.city) + ";" + row.date.strftime("%d/%m/%Y, %H:%M:%S")
        
        trips_id = set(filtered_data["trip_id"]) - set(layers[-1]["trip_id"])
        layer = liste_stop_times_ter.loc[(liste_stop_times_ter["trip_id"].isin(trips_id)) & date_mask]
        
        layer["previous_path"] = [paths[data.trip_id] for data in layer.itertuples()]
        layer = filter_direction_trip(layer, layers[-1]["city"].unique())

        layer = layer.sort_values(by="date").drop_duplicates(subset=["stop_id"])
        
        
        layer["current_path"] = layer["trip_id"] + ";" + layer["city"] + ";" + layer["date"].dt.strftime("%d/%m/%Y, %H:%M:%S")
        layers.append(layer)
        
        not_founded += 1
    return "not found excedeed 100 search"

In [51]:
depart = "HAGUENAU"
destination = "L'HERMITAGE"
shortest_path(depart, destination, "2024-11-19 05:00:00")


['OCESN830501F3492174:2024-10-11T00:34:04Z;STRASBOURG;19/11/2024, 05:50:00', 'OCESN839162F2879366:2024-10-10T10:27:58Z;PARIS;19/11/2024, 09:54:00', 'OCESN16753F3163270:2024-10-11T00:34:04Z;LE MANS;19/11/2024, 08:50:00', 'OCESN857691F3166965:2024-10-11T00:34:04Z;RENNES;19/11/2024, 08:19:00', "OCESN855505F3097753:2024-10-11T00:34:04Z;L'HERMITAGE;19/11/2024, 06:13:00"]


'destination trouver'